# Recurrent Neural Networks (RNN) using the Reservoir Computing (RC) paradigm for predicting the chaotic trajectories of the Lorenz system


This notebook is designed to evaluate your skills and ability to design a Recurrent Neural Network (RNN) to predict chaotic trajectories from the Lorenz system. We will use for this the so-called Reservoir Computing (RC) paradigm.

RNNs are a type of artificial neural network designed to work with sequential data. Unlike traditional feedforward neural networks, RNNs have internal connections that loop back, allowing information to persist across time steps. This makes RNNs particularly suited for tasks such as time series prediction, natural language processing, and sequence classification. In practice, a single sequence of $N$ elements will be considered as a single input to the RNN. A standard DNN would take that input as an $N$ size input and process all the elements at once in a single step. What changes in an RNN is that this single input is no longer processed in a single step, but the RNN loops over sequence elements internally.

An RNN consists of the following components:

Input Layer $\rightarrow$ Processes the sequential data.

Hidden Layer $\rightarrow$ Contains neurons with internal connections to model temporal dependencies.

Output Layer $\rightarrow$ Produces predictions.

The output at a time $t$ is defined as
$$
y_t = \mathbf{W}_y\cdot s_t + b_y
$$
with $\mathbf{W}_y$ the matrix of weights for the output, $b_y$ the bias for the output and $s_t$ the internal state at time $t$, given by
$$
s_t = \tanh\left(\mathbf{W}_x \cdot x_t + \mathbf{W}_s \cdot s_{t-1} + b_s\right)
$$
where $x_t$ is the input at time t, $s_{t-1}$ is the internal state at time $t-1$, $\mathbf{W}_x$ is the matrix of weights for the input, $\mathbf{W}_s$ is the matrix of weights for the internal state and $b_s$ is the bias for the internal state. Here, we have assumed a $\tanh$ activation function, but other types of activation function are possible. In this assignment, we will assume that $b_s=0$.

The main difficulty is to train the network to obtain the weights. In this notebook we will make use of a technique called Reservoir Computing (RC), which reduces the training to a simple linear regression. The RC method takes inspiration from the paradigm of the reservoir computers. In essence, it implies that the weights of the connected nodes of the matrix $\mathbf{W}_s$ and the elements of the matrix $W_x$ can be fixed randomly once for all, whereas the matrix $\mathbf{W}_y$ and the bias $b_y$ are computed by means of a simple linear regression, by solving the problem
$$
\mathbf{W}_y^*, b_y^* = \arg min\Vert \mathbf{W}_y\cdot s_t + b_y - y_t\Vert^2
$$
We will further simplify the problem by assuming that $b_y=b_y^*=0$.

Let us keep in mind that the previous equations are general and are used to predict $y_t$ from $x_t$. In our case, we want to predict the value of the time sequence at a time $t$ using the value of the time sequence at the previous step. Thus, the input $x_t$ is nothing else but the prediction at the previous step $y_{t-1}$.

The training of the RNN using the RC technique is therefore split into two simple steps, which makes it so powerful:
- First, the time sequence for the states $s_1, s_0, \dots, s_n$ is computed using the time sequence $x_0, x_1, \dots, x_{n-1}$, calculated from the Lorenz system. Note that to compute $s_t$, instead of using $x_t$, we use the value $x_{t-1}$ because the input at $t$ is actually the value of $x$ at the previous step.
- Second, the array $W_y$ is obtained via multivariate linear regression using the points $(s_1,y_0)$, $(s_2, y_1)$, ..., $(s_n,y_n)$, where $y_0=x_1$, $y_1=x_2$, and so on... (because the input $x$ for a given time corresponds to the output $y$ of the previous time).

After the training, we can use the matrices $W_x$, $W_s$ and $W_y$ to perform the predictions in an iterative way: given $y_n$ , we predict $y_{n+1}$ using as input $x_{n+1}=y_n$. For the next step, $y_{n+1}$ becomes the new input $x_{n+2}$ to predict $y_{n+2}$ and so on.  

IMPORTANT: note that in our case, the vectors $y$ and $x$ will have three dimensions, corresponding to the dimension of the Lorenz system. Therefore, in practice we want to predict the solution of the Lorenz system at time $t$, which is a 3D vector, from the solution of the Lorenz system at time $t-1$, which is also a 3D vector.

The use of linear regression reduces significantly the elapsed time for the training phase, and benefits from the absence of the vanishing gradient issue.

Follow the outlined steps, and fill in the missing code blocks where prompted.

First of all, let us import the required modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.linalg import eig

## 1. The Lorenz system

The Lorenz system is a set of three coupled differential equations:
$$\frac{dx}{dt} = a(y - x)$$
$$\frac{dy}{dt} = x(b - z) - y$$
$$\frac{dz}{dt} = xy - cz$$

These equations were derived for the first time by Lorenz in 1963, after simplifying a set of partial differential equations describing the motion of a fluid in between two layers of the atmosphere. Therefore, the parameters $a$, $b$ and $c$ have a real physical meaning, especially $b$, which correspond to the convection rate. For this assignment, we will impose $a=10$, $b=28$ and $c=8/3$ and the initial conditions $x(0) = 0.2$, $y(0) = -0.5$ and $z(0)=1$.
In this notebook, we solve the Lorenz equations using an explicit 4th order Runge-Kutta (RK4) scheme. The RK4 solver is the following

In [ ]:
def RK4(x1, x2, x3, a, b, c, dt):
    xold = np.array([x1, x2, x3])
    k1 = np.array([a * (x2 - x1), x1 * (b - x3) - x2, x1 * x2 - c * x3])
    k2 = np.array([a * (x2 + k1[1] * dt / 2 - (x1 + k1[0] * dt / 2)),
                   (x1 + k1[0] * dt / 2) * (b - (x3 + k1[2] * dt / 2)) - (x2 + k1[1] * dt / 2),
                   (x1 + k1[0] * dt / 2) * (x2 + k1[1] * dt / 2) - c * (x3 + k1[2] * dt / 2)])
    k3 = np.array([a * (x2 + k2[1] * dt / 2 - (x1 + k2[0] * dt / 2)),
                   (x1 + k2[0] * dt / 2) * (b - (x3 + k2[2] * dt / 2)) - (x2 + k2[1] * dt / 2),
                   (x1 + k2[0] * dt / 2) * (x2 + k2[1] * dt / 2) - c * (x3 + k2[2] * dt / 2)])
    k4 = np.array([a * (x2 + k3[1] * dt - (x1 + k3[0] * dt)),
                   (x1 + k3[0] * dt) * (b - (x3 + k3[2] * dt)) - (x2 + k3[1] * dt),
                   (x1 + k3[0] * dt) * (x2 + k3[1] * dt) - c * (x3 + k3[2] * dt)])
    
    return xold + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

We now define the parameters for the Lorenz system as well as the time vectors for training and testing. To emphasize the fact we want to predict the future of a time series, we use negative times for the training and positive times for the prediction.

In [ ]:
# Define time vectors for training and prediction
dt = 0.02
tmin_training = -100
time_training = np.arange(tmin_training, 0, dt)
nt_training = len(time_training)

tmax_prediction = 100
time_prediction = np.arange(dt, tmax_prediction + dt, dt)
nt_prediction = len(time_prediction)

### Question 0
Complete the following cell with the parameters and the initial conditions

In [ ]:
# Lorenz parameters
a = ...
b = ...
c = ...
# Initial conditions
x10 = ... #Initial condition for x
x20 = ... #Initial condition for y
x30 = ... #Initial condition for z

Let us now build the reservoir matrix $\mathbf{W}_s$, with a shape $N_s\times N_s$. This matrix represents the connections between all the possible pair of neurons. If the element $W_{s,i,j} = 0$, the neurons $i$ and $j$ are not connected. If the element $W_{s,i,j} = 1$, the neurons $i$ and $j$ are connected. The elements of this matrix will be initialized using a uniform distribution in $\left[-1,1\right]$. The elements of the reservoir matrix are re-scaled so that the largest magnitude of the eigenvalues (the so-called spectral radius) is equal to a desired value $\rho$. We will start with a spectral radius of $\rho=1.45$. In addition, the reservoir matrix will be built assuming it is a sparse Erdös-Rènyi network, with an average degree of $d=6$. This means that the rewiring probability $p=d/N_s$ is decreased if the size of the reservoir increases. This probability basically is the probability that two neurons are connected. The element $W_{s,i,j}$ is set to one with a probability $p$. In practice, we will make use of the built-in function erdos_renyi_graph from the networkx module. This function returns a matrix with zeros and ones. Once this matrix is built, the non-zero elements must therefore by multiplied by a value drawn from a uniform distribution in $[-1,1]$ and re-scaled so that the spectral radius is $\rho$.
We will build also the matrix $\mathbf{W}_x$. Its elements will also be initialized using a uniform distribution in $\left[-1, 1\right]$ and re-scaled to $\gamma=0.1$.

### Question 1
Complete the following cell to finalize the construction of $W_s$ and $W_x$

In [ ]:
# Parameters for Reservoir Computing
Ns = 300  # Number of nodes
rho = 1.45  # Spectral radius of the adjacency Erdos-Renyi matrix
d = 6  # Degree of the Erdos-Renyi matrix
p = d / Ns  # Rewiring probability
gamma = 0.1  # Input scaling

print(f'Parameters for the Reservoir Computing:')
print(f'Number of nodes Ns = {Ns}')
print(f'Rewiring probability p = {p}')
print(f'Degree of Erdos-Renyi matrix d = {d}')
print(f'Spectral radius of the Erdos-Renyi matrix rho = {rho}')

# Initialize matrices
Wx = np.zeros((Ns, 3))
G = nx.erdos_renyi_graph(Ns, p)
Ws = nx.to_numpy_array(G) #This matrix contains only 0 and 1.

# Generate random weights for matrices
for i in range(Ns):
    for j in range(3):
        Wx[i, j] = ...  # Random number between -gamma and gamma
    for j in range(Ns):
        Ws[i, j] = ... # Random weight between -1 and 1 ONLY for those elements that are non zero!!

# Re-scaling Ws so that rho(Ws) = rho
# Using eig() to get the spectral radius (max absolute eigenvalue)
eigenvalues, _ = eig(Ws)
max_spectral_radius = max(abs(eigenvalues))
Ws = ...

We will now compute the vector states $s$ from the data of the Lorenz system. For this purpose, we use the expression
$$
s_t = \tanh\left(\mathbf{W}_x \cdot x_t + \mathbf{W}_s \cdot s_{t-1} + b_s\right)
$$
with $b_s=0$. We also set the initial state $s_0$ to zero.
Be aware that in our case, the input at time $t$ is the solution of the Lorenz system at time $t-1$. This is because we want the predict the time $t$ from the time $t-1$, in other words, the output $y_t$ of the neural network is identified with the solution of the Lorenz system at time $t$ and the input $x_t$ of the neural network is identified with the solution of the Lorenz system at time $t-1$.

### Question 2
Complete the following cell to compute the state vectors for the training time window.

In [ ]:
# Initialize vectors for (x,y,z)
x = np.zeros((3, nt_training + nt_prediction))
x[0, 0] = ... #This represents x(t=0)
x[1, 0] = ... #This represents y(t=0)
x[2, 0] = ... #This represents z(t=0)
# Initialize the states as an array of zeros
state = np.zeros((Ns, nt_training + nt_prediction))
for it in range(1, nt_training):
    x1, x2, x3 = x[:, it - 1]
    t = time_training[it - 1]
    # Runge-Kutta 4 (RK4) to compute the ground truth
    x[:, it] = RK4(x1, x2, x3, a, b, c, dt)
    # Update state using the ground truth at time t-1 and the state at time t-1!!
    state[:, it] = np.tanh(...)

Now we will train the network using the simple linear regression to compute the matrix $\mathbf{W}_y$ and the bias vector $b_y$.

### Question 3
Complete the following cell to compute Wy using the simple multivariate linear regression formula that we saw in the lecture. Compute also the error in the training phase. For this you will have to compute the predictions using the state vectors.

In [ ]:
Wy = np.dot(......, ......)
#Initialize the prediction (called y) for the Lorenz system as an array of zeros.
y = np.zeros((3, nt_training + nt_prediction))
y[:, 0] = x[:, 0] #Except for t=0, where both vectors x and y must be equal
# Calculate error
error = 0
for it in range(1, nt_training):
    y[:, it] = Wy @ state[:, it] #This is the prediction from the state
    error += ...

As a final step, we will use the computed $W_y$ to make predictions in an iterative way, i.e. from a given state $s_t$, we compute the prediction $y_t$. This prediction will then be used as an input for the next step and so on.

### Question 4
Complete the following cell to make the predictions in the prediction time window in an iterative way, i.e. given (x,y,z) at time $t$, predict (x,y,z) at time $t+1$, then use the prediction at $t+1$ to predict the solution at $t+2$ and so on...

In [ ]:
# Prediction phase
for it in range(nt_training, nt_training + nt_prediction):
    state[:, it] = np.tanh(...)
    y[:, it] = ...
    # Use Runge-Kutta 4 (RK4) to compute the ground truth for the solution
    x1, x2, x3 = x[:, it - 1]
    x[:, it] = RK4(x1, x2, x3, a, b, c, dt)

It is now time to plot the results and see how the prediction matches the ground truth.
### Question 5
Plot both ground truth and prediction for the whole time window training+prediction. Afterwards, plot just a narrow time window during the beginning of the prediction time. What can you conclude? Plot also the time evolution of the state vectors from the training to the prediction phases. What do you observe?

In [ ]:
...

### Question 6
Repeat the previous steps modifying the spectral radius. Try one case with smaller spectral radius and one case with larger spectral radius. Do you observe any differences?

In [ ]:
...